# **METHODE DE NOTATION POUR LE TRade & ENvironment Database (TREND)**

In [163]:
# Import des packages
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.lines as mlines
import matplotlib.transforms as mtransforms
import matplotlib.pyplot as plt
%matplotlib inline

# Dataframe

In [ ]:
df = pd.read_csv('source/trend2022.csv', header=0, sep=';', encoding='latin-1')
df

In [ ]:
for numero, nom_colonne in enumerate(df.columns):
    print(numero,":", nom_colonne)

In [ ]:
trend = df.drop(df.columns[1:21], axis = 1)
trend

# Introduction de nouveaux individus

In [167]:
new_individual = pd.DataFrame([1] * trend.shape[1], index=trend.columns).T

# Ajout de cette ligne à la fin du DataFrame
trend = pd.concat([trend, new_individual], ignore_index=True)

In [168]:
new_individual = pd.DataFrame([0] * trend.shape[1], index=trend.columns).T

# Ajouter cette ligne à la fin du DataFrame
trend = pd.concat([trend, new_individual], ignore_index=True)

In [ ]:
trend

In [ ]:
trend.dtypes

In [ ]:
trend.isna().sum()

In [ ]:
trend.describe()

In [ ]:
summary = trend.describe()

var = summary.loc['mean'][summary.loc['mean'] > 0.4]

print("Variables avec une moyenne supérieure à 0.4 :")
for variable, mean in var.items():
    print(f"{variable}: {mean}")

# Score

## Score sur la Moyenne des domaines

In [ ]:
prefixes = [f'X{i}' for i in range(1, 16)]
new_df = pd.DataFrame()

# Pour chaque préfixe, la somme des colonnes correspondantes est:
for prefix in prefixes:
    cols_to_sum = [col for col in df.columns if col.startswith(prefix)]
    new_df[prefix] = trend[cols_to_sum].sum(axis=1)

new_df

In [ ]:
new_df.describe()

In [176]:
for prefix in prefixes:
    bad = new_df[prefix].min()
    good = new_df[prefix].max()
    new_df[f'{prefix}m'] = ((new_df[prefix] - bad) / (good - bad)) * (10 - 1) + 1

In [177]:
n_col = [f'{prefix}m' for prefix in prefixes]
new_df['note'] = np.sqrt(np.mean(np.square(new_df[n_col]), axis=1))
new_df['note'] = round(new_df['note'])
new_df['note'] = new_df['note'].astype(int)

In [ ]:
new_df

In [ ]:
ndf = pd.concat([trend['Trade.Agreement'], new_df['note']], axis = 1)
ndf = ndf.iloc[:-2]
ndf

In [ ]:
import matplotlib.pyplot as plt

# Histogramme pour visualiser la distribution
plt.hist(ndf['note'], bins=20, color='blue', edgecolor='black')

# Labels et un titre
plt.xlabel('IAE')
plt.ylabel('Effectif')
plt.title('IAE')

plt.show()

In [ ]:
# Table de distribution de la variable note
distribution_table = ndf['note'].value_counts().sort_index().reset_index()
distribution_table.columns = ['Valeur', 'Fréquence']

print(distribution_table)

# Data extraction

## Data par Accord de commerce

In [182]:
trend['Total'] = trend.iloc[:, 1:].sum(axis=1)

In [ ]:
df_slice = df.iloc[:, 3:21]
df_scores = pd.concat([df['Trade.Agreement'], df_slice, trend['Total'], ndf['note']], axis=1)
df_scores.rename(columns={'Total': 'Nombre de normes'}, inplace=True)
df_scores.rename(columns={'note': 'IAE'}, inplace=True)
df_scores

In [ ]:
dx = pd.read_csv('source/infos_accords.csv')
dx = dx.drop(columns=['depth_index', 'mar_typedepth'])
df_scores = pd.merge(df_scores, dx, on = ['Trade.Agreement'], how = 'left')
df_scores.head(10)

In [ ]:
df_scores.to_csv("results/df_score.csv", index=False)
print("Le fichier csv a été sauvegardé avec succès.")

## Data par pays

In [ ]:
data_pays = pd.read_csv('source/trenddyadic2022.csv', header=0, sep=';', encoding='latin-1')
data_pays = data_pays.drop(data_pays.columns[24:322], axis = 1)
data_pays

In [ ]:
data_pays = pd.merge(data_pays, df_scores[['Trade.Agreement', 'Nombre de normes', 'IAE']], on='Trade.Agreement', how='left')
data_pays

In [188]:
df1 = data_pays.drop(columns=['country2'])
df2 = data_pays.drop(columns=['country1'])


In [189]:
df1.rename(columns={'country1': 'country'}, inplace=True)
df2.rename(columns={'country2': 'country'}, inplace=True)
data_pays = pd.concat([df1, df2], ignore_index=True)

In [ ]:
data_pays = data_pays.drop_duplicates(subset=['Trade.Agreement', 'country'])
data_pays

In [ ]:
country = data_pays.copy()
country

In [ ]:
country = country.drop(country.columns[0], axis = 1)
country = country.drop(country.columns[1:15], axis = 1)
country = country.drop(country.columns[2:8], axis = 1)
country

In [ ]:
country.to_excel('country_trend.xlsx', sheet_name = 'country score', index = False)
print('Well successed !')

**Il y a un problème avec les noms des pays. Ils ne sont pas conforme pour notre analyse. A l'aide de Excel** (`VLOOKUP function`) **, ils sont donc corrigés avant utilisation**

**!!! Régler le problème des noms de pays avant de continuer en utlisant *'res_pays problems.xlsx'*, pour corriger dans *'country_trend.xlsx'* et en appelant la correction 'C'**

In [ ]:
country = pd.read_excel('source/country_trend2.xlsx', sheet_name='country score')
country

In [195]:
country['country'] = country['C']
country = country.drop(columns='C', axis = 1)
country.rename(columns={'Note':'IAE'}, inplace=True)

In [ ]:
country

In [ ]:
moy2 = country.groupby('country')['IAE'].mean().reset_index()
moy2 = moy2.rename(columns={'IAE': 'IAE moyen'})
moy2

In [ ]:
som = country.groupby('country')['Nombre de normes'].sum().reset_index()
som = som.rename(columns={'Nombre de normes': 'Nombre de normes signées'})
som['Nombre de normes signées'] = som['Nombre de normes signées'].astype(int)
som

In [ ]:
max = country.groupby('country')['Year'].max().reset_index()
max = max.rename(columns={'Year': 'YearLastPTASigned'})
max

In [ ]:
min = country.groupby('country')['Year'].min().reset_index()
min = min.rename(columns={'Year': 'YearFirstPTASigned'})
min

In [ ]:
country_trend = pd.concat([min, max['YearLastPTASigned'], som['Nombre de normes signées'], moy2['IAE moyen']], axis=1)
country_trend

In [ ]:
country_trend.describe()

In [ ]:
country_trend.to_csv('results/data_pays.csv', index = False)
print('Well done !')